In [5]:
# df = pd.read_pickle("/Users/mario/Downloads/Graph_State_Uni.pkl")

In [31]:
# df_clean = df.dropna(subset=['Graph'])

In [55]:
# df_clean

,Experiment,Town,County,State,Treatment,Search_Place,Graph
0,"(AlAMU, 1875)",Huntsville,Madison,Alabama,1,"Huntsville, Alabama","(56922621, 56923776, 56923781, 56923787, 56924..."
1,"(AlAMU, 1875)",Montgomery,Montgomery,Alabama,0,"Montgomery, Alabama","(58878326, 58878327, 58878330, 58878370, 58878..."
3,"(AubU, 1856)",Auburn,Lee,Alabama,1,"Auburn, Alabama","(56844738, 56844741, 56845079, 56845082, 56845..."
4,"(AubU, 1856)",Florence,Lauderdale,Alabama,0,"Florence, Alabama","(56540270, 56540280, 56540305, 56540703, 56540..."
5,"(AubU, 1856)",Talladega,Talladega,Alabama,0,"Talladega, Alabama","(52315445, 52318540, 52326293, 52387532, 52398..."
...,...,...,...,...,...,...,...
995,"(UWi, 1838)",Madison,Dane,Wisconsin,1,"Madison, Wisconsin","(29981753, 29983388, 30004443, 30052852, 30052..."
996,"(UWi, 1838)",Ripon,Fond du Lac,Wisconsin,0,"Ripon, Wisconsin","(230969615, 230969647, 230969653, 230972997, 2..."
997,"(UWy, 1886)",Laramie,Albany,Wyoming,1,"Laramie, Wyoming","(156300705, 156300836, 156300840, 156300844, 1..."
998,"(UWy, 1886)",Cheyenne,Laramie,Wyoming,0,"Cheyenne, Wyoming","(157345948, 157345956, 157345970, 157346034, 1..."


In [3]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import pandas as pd
import networkx as nx
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader
import ast
from torch_geometric.loader import DataLoader

In [41]:
df["Graph"][0]

import torch
from torch_geometric.data import Data
import networkx as nx

def process_graph(row):
    # Check if the graph is a valid MultiDiGraph object
    G = row['Graph']
    
    if isinstance(G, nx.MultiDiGraph):
        edge_index = torch.tensor(list(G.edges)).t().contiguous()
        
        # Assuming node features are an identity matrix (one-hot encoding)
        x = torch.eye(G.number_of_nodes())
        
        # Get the treatment label
        y = torch.tensor([row['Treatment']], dtype=torch.long)
        
        return Data(x=x, edge_index=edge_index, y=y)
    else:
        # Handle cases where the graph is invalid or missing
        return None

# Applying the function to the DataFrame
graphs = [process_graph(row) for idx, row in df.iterrows()]

# Remove None values from the list
graphs = [g for g in graphs if g is not None]



# Train-test split
train_data, test_data = train_test_split(graphs, test_size=0.2)

train_loader = DataLoader(train_data, batch_size=32)
test_loader = DataLoader(test_data, batch_size=32)



## to adjust for node features matrix being to large to fit into memory

In [52]:
def process_graph(row):
    G = row['Graph']
    
    if isinstance(G, nx.MultiDiGraph):
        edge_index = torch.tensor(list(G.edges)).t().contiguous()
        
        # Node features are simply node indices instead of one-hot encoding
        num_nodes = G.number_of_nodes()
        node_indices = torch.arange(num_nodes).unsqueeze(1)
        
        y = torch.tensor([row['Treatment']], dtype=torch.long)
        
        return Data(x=node_indices, edge_index=edge_index, y=y)
    else:
        return None

# Apply the function to the DataFrame
graphs = [process_graph(row) for idx, row in df_clean.iterrows()]

# Remove None values from the list
graphs = [g for g in graphs if g is not None]


## Shows Node Structure

In [ ]:
# for idx, data in enumerate(graphs):
#    print(f"Graph {idx}: Nodes = {data.num_nodes}, Edges = {data.edge_index.size()}") 

In [7]:
# Model Definition
class GCN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_features, hidden_features)
        self.conv2 = GCNConv(hidden_features, out_features)
        self.clf = nn.Linear(out_features, 2)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = self.clf(x)
        x = F.log_softmax(x, dim=1)
        return x

In [68]:
# Training the Model
model = GCN(in_features=9000, hidden_features= 64, out_features= 2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


# Change for results
epochs = 5

for epoch in range(epochs):
    model.train()
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 3 but got size 2 for tensor number 1 in the list.

In [64]:
def process_graph(row):
    G = nx.Graph()  # Use Graph for undirected or DiGraph for directed graphs

    # Assuming 'row['Graph']' is a list of edges (e.g., [(src1, dst1), (src2, dst2), ...])
    edges = ast.literal_eval(row['Graph'])

    G.add_edges_from(edges)
    edge_index = torch.tensor(list(G.edges)).t().contiguous()

    if edge_index.size(0) != 2:
        print("Unexpected edge index size:", edge_index.size())
    
    # Process node features and labels as appropriate
    return Data(x=..., edge_index=edge_index, y=...)


In [ ]:
# Testing the Model
model.eval()
correct = 0
for data in test_loader:
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    correct += (pred == data.y).sum().item()

accuracy = correct / len(test_loader.dataset)
print(f'Accuracy: {accuracy:.4f}')


In [9]:
model_dict = torch.load("data/training/graphcnn_model.pth")

In [10]:
model = GCN(in_features=9000, hidden_features= 64, out_features= 2)
model.load_state_dict(model_dict)

RuntimeError: Error(s) in loading state_dict for GCN:
	Missing key(s) in state_dict: "conv1.bias", "conv1.lin.weight", "conv2.bias", "conv2.lin.weight", "clf.weight", "clf.bias". 
	Unexpected key(s) in state_dict: "gcn1.linear.weight", "gcn1.linear.bias", "gcn2.linear.weight", "gcn2.linear.bias", "attention_pool.attention_dense.weight", "attention_pool.attention_dense.bias", "attention_pool.score_dense.weight", "attention_pool.score_dense.bias", "fc1.weight", "fc1.bias", "fc2.weight", "fc2.bias". 